In [ ]:
import os
import sys
import copy
import numpy as np
import matplotlib.pyplot as plt
import h5py
import pickle
from qiskit import IBMQ
from qiskit_experiments.database_service import DbExperimentDataV1 as DbExperimentData

sys.path.append(os.path.dirname(os.getcwd()))
from observables import plot_counts_with_curve
from calibrations import MeasurementErrorExperiment

In [1]:
provider = IBMQ.load_account()

backend = provider.get_backend('ibm_lagos')
physical_qubits = [1, 2]
mem_experiment_id = ''

use_runtime = True

NameError: name 'IBMQ' is not defined

In [ ]:
if not mem_experiment_id and not backend.configuration().simulator:
    exp = MeasurementErrorExperiment(physical_qubits, circuits_per_state=(backend.configuration().max_experiments // 32))
    exp_data = exp.run(backend=backend, shots=8192)
    print(exp_data.experiment_id)
    exp_data.block_for_results()

    if backend.name() == 'ibm_kawasaki':
        mem_experiment_id = 'mem_' + '_'.join(map(str, physical_qubits))
        with h5py.File('mem_kawasaki.h5', 'a') as out:
            try:
                out[mem_experiment_id] = exp_data.analysis_results()[0].value
            except KeyError:
                out.create_dataset(mem_experiment_id, data=exp_data.analysis_results()[0].value)
    else:
        mem_experiment_id = exp_data.experiment_id
        exp_data.save()

In [ ]:
if backend.configuration().simulator:
    error_matrix = None
elif backend.name() == 'ibm_kawasaki':
    with h5py.File('mem_kawasaki.h5', 'r') as source:
        error_matrix = source[mem_experiment_id][:]
else:
    exp_data = DbExperimentData.load(mem_experiment_id, provider.service("experiment"))
    analysis_result = exp_data.analysis_results()[0]
    error_matrix = analysis_result.value

In [ ]:
num_sites = 2
num_tsteps = 8
tsteps_per_rqd = 2
num_rqd_steps = num_tsteps // tsteps_per_rqd

program_inputs = {
    'num_sites': num_sites,
    'aJ': 1.,
    'am': 0.5,
    'omegadt': 0.2,
    'num_tsteps': num_tsteps,
    'tsteps_per_rqd': tsteps_per_rqd,
    'physical_qubits': physical_qubits,
    'minimizer_shots_per_job': 10000,
    'minimizer_jobs': 20,
    'forward_shots': 40000,
    'max_sweeps': 100
}

if not backend.configuration().simulator:
    program_inputs['api_token'] = provider.credentials.access_token
    program_inputs['error_matrix'] = error_matrix
    
if program_inputs['num_sites'] == 2:
    num_ansatz_params = 2
elif program_inputs['num_sites'] == 4:
    num_ansatz_params = 10

In [ ]:
results_file_name = f'{backend.name()}_{"_".join(map(str, physical_qubits))}.h5'
state_file_name = f'{backend.name()}_{"_".join(map(str, physical_qubits))}_state.pkl'

with h5py.File(results_file_name, 'w') as output:
    output.create_dataset('forward_counts', (num_tsteps, 2 ** num_sites), dtype='f8')
    output.create_dataset('num_sweeps', (num_rqd_steps,), dtype='i')
    output.create_dataset('shots', (num_rqd_steps, program_inputs['max_sweeps']), dtype='i')
    output.create_dataset('costs', (num_rqd_steps, program_inputs['max_sweeps']), dtype='f8')
    output.create_dataset('optimal_params', (num_rqd_steps, num_ansatz_params), dtype='f8')
    
def interim_result_callback(job_id, interim_result):
    rqd_step = interim_result['rqd_step']
    
    if 'forward_counts' in interim_result:
        # One call per RQD step - before minimization
        print(f'Step: {rqd_step} Forward counts: {interim_result["forward_counts"]}')
        
        forward_counts = np.zeros((tsteps_per_rqd, 2 ** num_sites), dtype='f8')
        for itstep, counts in enumerate(interim_result['forward_counts']):
            for key, value in counts.items():
                forward_counts[itstep, int(key, 2)] = value

        start = rqd_step * tsteps_per_rqd
        with h5py.File(results_file_name, 'a') as output:
            output['forward_counts'][start:start + tsteps_per_rqd] = forward_counts
            
    elif 'optimal_params' in interim_result:
        # One call per RQD step - after minimization
        print(f'Step: {rqd_step} Optimal params: {interim_result["optimal_params"]}')
        
        with h5py.File(results_file_name, 'a') as output:
            output['optimal_params'][rqd_step] = interim_result['optimal_params']
            num_sweeps = len(interim_result['shots_values'])
            output['num_sweeps'][rqd_step] = num_sweeps
            output['shots'][rqd_step, :num_sweeps] = interim_result['shots_values']
            output['costs'][rqd_step, :num_sweeps] = interim_result['cost_values']

    else:
        # One call per sweep
        print(f'Step: {rqd_step} Sweep: {interim_result["state"]["sweep"]} Cost: {interim_result["state"]["cost"]} Shots: {interim_result["state"]["shots"]}')

        resume_from = {
            'rqd_step': interim_result['rqd_step'],
            'state': interim_result['state']
        }
        if 'minimizer_state' in interim_result:
            resume_from['minimizer_state'] = interim_result['minimizer_state']
        
        with open(state_file_name, 'wb') as out:
            pickle.dump(resume_from, out)

In [ ]:
if use_runtime:
    while True:
        options = {'backend_name': backend.name()}
        job = provider.runtime.run(
            program_id='schwinger-rqd-oGExmQ8LjP',
            options=options,
            inputs=program_inputs,
            callback=interim_result_callback)

        print(f"job creation time: {job.creation_date}")
        print(f"job id: {job.job_id()}")

        try:
            message = job.result()
            print(message)
            break
        except Exception as ex:
            sys.stdout.write(ex.args[0])
        except:
            raise

        if 'ran longer than maximum execution time.' in job.error_message():
            try:
                with open(state_file_name, 'rb') as source:
                    program_inputs['resume_from'] = pickle.load(source)
            except FileNotFoundError:
                pass

        else:
            raise RuntimeError('failed')
            
else:
    class UserMessenger:
        def publish(
                self,
                message,
                encoder=None,
                final=False
        ):
            interim_result_callback(message)
        
    from runtime.main import main
    
    user_messenger = UserMessenger()
    main(backend, user_messenger, **program_inputs)